In [0]:
import sys
import itertools

In [0]:
#function to number of mutual friends between a pair of users.
def friends_mapping_mutual(iterate):
    mutual = []
    userid = iterate[0]
    friends = iterate[1]

    for friendid in friends:
        key = (userid, friendid)
        if userid > friendid:
            key = (friendid, userid)
        mutual.append((key, 0))

    for pairof_friend in itertools.combinations(friends, 2):
        friend0 = pairof_friend[0]
        friend1 = pairof_friend[1]
        key = (friend0, friend1)
        if friend0 > friend1:
            key = (friend1, friend0)
        mutual.append((key, 1))
    return mutual 

In [0]:
# splitting each line in the text file into user id and friends.
def Spliting_friends(line):
    split = line.split()
    userid = int(split[0])

    if len(split) == 1:
        friends = []
    else:
        friends = list(map(lambda x: int(x), split[1].split(",")))
    return userid, friends

In [0]:
#Function to recommend friends based on number of mutual friends
def count_mutual_friend_to_recommend(mutl):
    mutuals = mutl[0]
    count = mutl[1]
    friend1 = mutuals[0]
    friend2 = mutuals[1]

    recommendfriends1 = (friend1, (friend2, count))
    recommendfriends2 = (friend2, (friend1, count))
    return [recommendfriends1, recommendfriends2]

In [0]:
#Function to sort the first best 10 recommendations for each user.
def sorting_recommendation(suggestions):
    if len(suggestions) > 1024:
        highest_indices = []

        for suggestion_number in range(0, 10):
            present_high_index = 0
            for l in range(1, len(suggestions)):
                recommend = suggestions[l]
                if recommend[1] >= suggestions[present_high_index][1] and l not in highest_indices:
                    present_high_index = l
            highest_indices.append(present_high_index)
        suggestions = [suggestions[l] for l in highest_indices]

    suggestions.sort(key=lambda x: (-x[1], x[0]))
    return list(map(lambda x: x[0], suggestions))[:10]

In [0]:
common = sc.textFile("/FileStore/tables/soc_LiveJournal1Adj.txt")

friend_mapping = common.map(Spliting_friends)
friend_list = friend_mapping.flatMap(friends_mapping_mutual)
friend_list.cache()
count_mutual_friend = (friend_list.groupByKey().filter(lambda edge: 0 not in edge[1]).map(lambda edge: (edge[0], sum(edge[1]))))
mutual_recommendations = (count_mutual_friend.flatMap(count_mutual_friend_to_recommend).groupByKey().map(lambda mutl: (mutl[0], sorting_recommendation(list(mutl[1])))))

#display output in the format [(user, [list of top 10 friends recommendation based on number of mutual friends])
mutual_recommendations.collect()

Out[24]: [(15122, [17069, 17436, 26872, 2680, 134, 8501, 17038, 17450, 29845, 43175]),
 (13976,
  [13998, 14026, 13982, 14101, 14191, 14218, 23672, 14174, 14134, 14203]),
 (38246, [1356, 6531, 6542, 6654, 6976, 7022, 7029, 20045, 37861, 38207]),
 (3450, [29003, 14867, 17450, 2036, 4590, 11611, 15750, 27484, 29719, 20829]),
 (24804, [211, 351, 452, 897, 909, 956, 986, 1114, 1383, 1390]),
 (534, [605, 615, 7509, 523, 524, 526, 527, 531, 535, 536]),
 (35116, [35071, 498, 1581, 2782, 4687, 7639, 8667, 9607, 10052, 12577]),
 (41460, [16508, 22981, 35635, 41407, 41435, 41456, 41474, 41573, 45837, 112]),
 (26164, [2140, 2163, 2210, 2211, 2216, 2219, 2241, 2262, 2280, 2319]),
 (34408, [34150, 34176, 34207, 626, 3574, 3575, 7506, 11393, 11401, 11404]),
 (7092, [6487, 6955, 6980, 6542, 7059, 12700, 3401, 6578, 6976, 6994]),
 (22736, [22737, 22741, 15697, 22732, 22744, 22745, 3077, 3101, 4048, 5429]),
 (25344,
  [47712, 5803, 15527, 47713, 47714, 47717, 47718, 47722, 47723, 47728]),
 (36476, [364